In [2]:
import pandas as pd
import numpy as np
import sys, os
# Add the parent directory to the sys.path list
sys.path.append(os.path.abspath('../'))
import NAS.utils as utils
import math

In [73]:
from reservoirpy.nodes import (Reservoir, Ridge, IPReservoir, NVAR, RLS, Input)
import numpy as np
from reservoirpy.datasets import narma, lorenz96
from reservoirpy.observables import rmse, rsquare

X = narma(n_timesteps=2000)

sampleX = np.random.rand(10000, 1)

reservoir1 = Reservoir(units=500, lr=0.3, sr=1.25)
reservoir2 = Reservoir(units=500, lr=0.3, sr=1.25)
readout = Ridge(output_dim=1, ridge=1e-5)
# reservoir2 <<= readout
esn = reservoir1 >> reservoir2 >> readout
def func():
    esn.fit(X[:500], X[1:501], warmup=100, force_teachers=True)

memoryUsed = measure_memory_usage(func)
print(memoryUsed)
predictions = esn.run(X[501:-1])
nodeNames = [node.name for node in esn.nodes]
# predictions = predictions[nodeNames[-1]]
print("RMSE:", rmse(X[502:], predictions), "R^2 score:", rsquare(X[502:], predictions))


11.166996002197266
RMSE: 0.6330188775973445 R^2 score: -48.39569117317432


In [2]:
def getData():
    df = pd.read_csv("../data/electricity.csv", index_col="Unnamed: 0")
    df.index = pd.to_datetime(df['Date'] + " " + df['Time'])
    df = df.sort_index()
    df = df.iloc[-100000:]

    df['Target_active_power'] = df['Global_active_power'].shift(-1)
    df = df.replace("?", np.nan).ffill().dropna()

    T = len(df)
    valid_boundary = 0.6
    test_boundary = 0.8
    valid_index = int(T*valid_boundary)
    test_index = int(T*test_boundary)
    train = df.iloc[:valid_index, 2:].to_numpy().astype(float)
    val = df.iloc[valid_index:test_index, 2:].to_numpy().astype(float)
    test = df.iloc[test_index:, 2:].to_numpy().astype(float)
    trainX = train[:, :-1]
    trainY = train[:, -1:]
    valX = val[:, :-1]
    valY = val[:, -1:]
    testX = test[:, :-1]
    testY = test[:, -1:]
    return trainX, trainY, valX, valY, testX, testY

trainX, trainY, valX, valY, testX, testY = getData()

In [10]:
def getData():
    water = pd.read_csv("../datasets/Water.csv").to_numpy()
    trainLen = math.floor(len(water)*0.5)
    valLen = math.floor(len(water)*0.7)
    
    train_in = water[0:trainLen, :18]
    train_out = water[0:trainLen, 18:]
    val_in = water[trainLen:valLen, :18]
    val_out = water[trainLen:valLen, 18:]
    test_in = water[valLen:, :18]
    test_out = water[valLen:, 18:]
    return train_in, train_out, val_in, val_out, test_in, test_out

trainX, trainY, valX, valY, testX, testY = getData()

In [4]:
from NAS.memory_estimator import measure_memory_usage

architecture = utils.generateRandomArchitecture(trainX.shape[1], trainY.shape[1], trainX.shape[0], memoryLimit=1024)
expected = utils.estimateMemory(architecture, trainX.shape[0])

def func():
    model = utils.constructModel(architecture)
    utils.trainModel(model, trainX, trainY)
actual = measure_memory_usage(func)
print(actual, expected)

1171.9860248565674 837.9480812072754


In [19]:
combinedData = np.hstack((X.iloc[:, 1:].to_numpy(), y.to_numpy()))
combinedData.shape

(19735, 28)

In [20]:
def getData():
    from ucimlrepo import fetch_ucirepo
    appliances_energy_prediction = fetch_ucirepo(id=374)
    X = appliances_energy_prediction.data.features.iloc[:, 1:].to_numpy()
    y = appliances_energy_prediction.data.targets.to_numpy()

    trainLen = 14000
    valLen = 16000
    
    train_in = X[0:trainLen]
    train_out = y[0:trainLen]
    val_in = X[trainLen:valLen]
    val_out = y[trainLen:valLen]
    test_in = X[valLen:]
    test_out = y[valLen:]
    return train_in, train_out, val_in, val_out, test_in, test_out

trainX, trainY, valX, valY, testX, testY = getData()

In [44]:
from reservoirpy.datasets._seed import get_seed
from reservoirpy.utils.random import rand_generator
from reservoirpy.utils.validation import check_vector

def narma(
    n_timesteps,
    order = 30,
    a1 = 0.2,
    a2 = 0.04,
    b = 1.5,
    c = 0.001,
    x0 = [0.0],
    seed = None,
) -> np.ndarray:
    """Non-linear Autoregressive Moving Average (NARMA) timeseries,
    as first defined in [14]_, and as used in [15]_.

    NARMA n-th order dynamical system is defined by the recurrent relation:

    .. math::

        y[t+1] = a_1 y[t] + a_2 y[t] (\\sum_{i=0}^{n-1} y[t-i]) + b u[t-(
        n-1)]u[t] + c

    where :math:`u[t]` are sampled following a uniform distribution in
    :math:`[0, 0.5]`.

    Parameters
    ----------
    n_timesteps : int
        Number of timesteps to generate.
    order: int, default to 30
        Order of the system.
    a1 : float, default to 0.2
        :math:`a_1` parameter of the system.
    a2 : float, default to 0.04
        :math:`a_2` parameter of the system.
    b : float, default to 1.5
        :math:`b` parameter of the system.
    c : float, default to 0.001
        :math:`c` parameter of the system.
    x0 : array-like of shape (init_steps,), default to [0.0]
        Initial conditions of the system.
    seed : int or :py:class:`numpy.random.Generator`, optional
        Random state seed for reproducibility.

    Returns
    -------
    array of shape (n_timesteps, 1)
        NARMA timeseries.

    References
    ----------
    .. [14] A. F. Atiya and A. G. Parlos, ‘New results on recurrent
           network training: unifying the algorithms and accelerating
           convergence,‘ in IEEE Transactions on Neural Networks,
           vol. 11, no. 3, pp. 697-709, May 2000,
           doi: 10.1109/72.846741.

    .. [15] B.Schrauwen, M. Wardermann, D. Verstraeten, J. Steil,
           D. Stroobandt, ‘Improving reservoirs using intrinsic
           plasticity‘,
           Neurocomputing, 71. 1159-1171, 2008,
           doi: 10.1016/j.neucom.2007.12.020.
    """
    if seed is None:
        seed = get_seed()
    rs = rand_generator(seed)

    y = np.zeros((n_timesteps + order, 1))

    x0 = check_vector(np.atleast_2d(np.asarray(x0)))
    y[: x0.shape[0], :] = x0

    # noise = rs.uniform(0, 0.5, size=(n_timesteps + order, 1))
    noise = np.random.uniform(0, 1, (n_timesteps + order, 1))
    for t in range(order, n_timesteps + order - 1):
        print(a1 * y[t], a2 * y[t], np.sum(y[t - order+1 : t+1]), b * noise[t - order], noise[t])
        y[t + 1] = (a1 * y[t] + a2 * y[t] * np.sum(y[t - order+1 : t+1])+ b * noise[t - order] * noise[t]+ c)
    return y[order:, :]

In [15]:
import pandas as pd
import math
import numpy as np

def getDataSingleCol():
    water = pd.read_csv("../datasets/Water.csv").to_numpy()
    firstRow = water[0, :]
    lastCol = water[1:, -1]
    allData = np.concatenate((firstRow, lastCol))
    
    trainLen = math.floor(len(water)*0.5)
    valLen = math.floor(len(water)*0.7)
    
    train_in = lastCol[0:trainLen]
    train_out = lastCol[0:trainLen]
    val_in = lastCol[trainLen:valLen]
    val_out = lastCol[trainLen:valLen]
    test_in = lastCol[valLen:]
    test_out = lastCol[valLen:]
    return train_in, train_out, val_in, val_out, test_in, test_out, water
getDataSingleCol()

(array([0.72093, 0.68895, 0.625  , ..., 0.69186, 0.63953, 0.61628]),
 array([0.72093, 0.68895, 0.625  , ..., 0.69186, 0.63953, 0.61628]),
 array([0.60465, 0.62791, 0.60174, ..., 0.77616, 0.81105, 0.7907 ]),
 array([0.60465, 0.62791, 0.60174, ..., 0.77616, 0.81105, 0.7907 ]),
 array([0.74419, 0.70349, 0.67733, ..., 0.6686 , 0.57849, 0.51453]),
 array([0.74419, 0.70349, 0.67733, ..., 0.6686 , 0.57849, 0.51453]),
 array([[0.52326, 0.46512, 0.42442, ..., 0.8343 , 0.84593, 0.79651],
        [0.46512, 0.42442, 0.41279, ..., 0.84593, 0.79651, 0.72093],
        [0.42442, 0.41279, 0.40407, ..., 0.79651, 0.72093, 0.68895],
        ...,
        [0.77616, 0.72965, 0.69767, ..., 0.74419, 0.72384, 0.6686 ],
        [0.72965, 0.69767, 0.67442, ..., 0.72384, 0.6686 , 0.57849],
        [0.69767, 0.67442, 0.68023, ..., 0.6686 , 0.57849, 0.51453]]))

In [1]:
import numpy as np

def adjacency_matrix_to_list_pairs(adj_matrix):
    edge_list = []
    num_nodes = len(adj_matrix)
    
    for i in range(num_nodes):
        for j in range(num_nodes):
            if adj_matrix[i][j] == 1:
                edge_list.append([i, j])
    
    return edge_list

def generate_dag_adjacency_matrix(num_nodes):
    # Create an empty adjacency matrix
    adj_matrix = np.zeros((num_nodes, num_nodes), dtype=int)
    
    # Populate the adjacency matrix such that there are no cycles
    for i in range(num_nodes - 1):  # Adjust the range to stop before the last node
        # Nodes can only connect to subsequent nodes to ensure no cycles
        for j in range(i + 1, num_nodes):
            # Introduce randomness in connections; not every node connects to every other node
            if np.random.rand() > 0.5:
                adj_matrix[i, j] = 1
                
    # Ensure there's at least one path from start (0) to end (num_nodes-1)
    # Directly connect the start node to some node if not already connected
    if not adj_matrix[0].any():
        adj_matrix[0, np.random.randint(1, num_nodes - 1)] = 1  # Avoid connecting directly to the last node

    # Ensure the last node can be reached from some node but does not have outgoing edges
    if not adj_matrix[:, num_nodes - 1].any():
        adj_matrix[np.random.randint(0, num_nodes - 1), num_nodes - 1] = 1

    return adjacency_matrix_to_list_pairs(adj_matrix)

generate_dag_adjacency_matrix(7)

[[0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [1, 2],
 [1, 4],
 [1, 6],
 [2, 4],
 [2, 5],
 [2, 6],
 [3, 5],
 [4, 5],
 [5, 6]]

In [114]:
import networkx as nx
import random

def createRandomGraph(numNodes):
    while True:
        G = nx.gnp_random_graph(numNodes - 2, 1, directed=True)
        DAG = nx.DiGraph([(u, v, {'weight':random.randint(-10, 10)}) for (u, v) in G.edges() if u<v])
        if len(DAG.nodes)==numNodes-2:
            break
    
    sources = []
    sinks = []
    for node in DAG.nodes:
        hasPredecessor = False
        for otherNode in DAG.nodes:
            if DAG.has_predecessor(node, otherNode):
                hasPredecessor = True
        if not hasPredecessor:
            sources.append(node)
            
        hasSuccessor = False
        for otherNode in DAG.nodes:
            if DAG.has_successor(node, otherNode):
                hasSuccessor = True
        if not hasSuccessor:
            sinks.append(node)

    edges = [[0, source+1] for source in sources] + [[e1 + 1, e2 + 1] for (e1, e2) in DAG.edges] + [[sink+1, numNodes-1] for sink in sinks]
    return edges
    

In [123]:
createGraph(9)

True
[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 4), (1, 5), (1, 6), (2, 3), (2, 4), (2, 6), (3, 4), (3, 5), (3, 6), (4, 5), (4, 6), (5, 6)] [0, 1, 2, 3, 4, 5, 6]


[[0, 1],
 [1, 2],
 [1, 3],
 [1, 4],
 [1, 5],
 [1, 6],
 [2, 3],
 [2, 5],
 [2, 6],
 [2, 7],
 [3, 4],
 [3, 5],
 [3, 7],
 [4, 5],
 [4, 6],
 [4, 7],
 [5, 6],
 [5, 7],
 [6, 7],
 [7, 8]]